In [27]:
import requests
import json
import pandas as pd
from datetime import datetime

# Point to correct subgraph URL
url = 'https://api.thegraph.com/subgraphs/name/decentraland/marketplace'
#url = 'https://gateway.thegraph.com/api/[api-key]/subgraphs/id/0x89fddab2f93417182cdcdb0b8b3322b93ab3a192-0'
url2 = 'https://api.thegraph.com/subgraphs/name/uniswap/uniswap-v2'

# initialize full_data for Query1 of Decentraland Dataset
df1_fulldata = pd.DataFrame()
df2_fulldata = pd.DataFrame()
df_data = pd.DataFrame()

for i in range(0,9) :
    # Set query (which uses text input to specify Decentraland data)
    query = '''query {
  orders(where:{status:sold, category: parcel}, orderBy: createdAt, orderDirection: asc, first:1000, skip: %s){
    id
    txHash
    price
    owner
    buyer
    status
    tokenId
    category
    createdAt
    nft{
      searchParcelX
      searchParcelY
    }
  }
}''' % (i*1000)


    # Make the request
    r = requests.post(url, json={'query': query})
    #print(r.status_code)
    #print(r.text)
    
    # JSON adjustment
    json_data = json.loads(r.text)
    
    # extract JSON to convert to a dataframe
    df_data = json_data['data']['orders']
    df1_fulldata = pd.DataFrame(df_data)
        
    df2_fulldata = pd.concat([df2_fulldata,df1_fulldata])
    

#Removing NULLS - Data Cleaning
df = df2_fulldata.dropna()

#print(df.isnull().values.any())
#print(df.isnull().sum().sum())

#Converting from Unix timestamp to datetime
df['createdAt'] = pd.to_datetime(df['createdAt'],unit='s')

#Splitting 'nft' column into 'x' and 'y'
df[['x','y']] = df.nft.apply(
  lambda z: pd.Series(str(z).split(",")))

df = df.drop(['nft'], axis=1)

#Removing extra data from columns : Cleaning
df['x'] = df['x'].map(lambda x: str(x)[19:-1])
df['y'] = df['y'].map(lambda y: str(y)[19:-2])

#Reducing 'Price' to usable int form in Mana (price/(10^18))
df['price_cut'] = df['price'].map(lambda y: str(y)[:-14])
df['price_cut'] = pd.to_numeric(df['price_cut'])
df['price_MANA'] = df['price_cut']/(10000)

#Extracting CreatedAt Date from Datetime
df['date'] = pd.to_datetime(df['createdAt']).dt.date

df1 = df.drop(['price','price_cut'], axis=1)

print(df1)

                                          buyer category           createdAt  \
0    0x1b39eafd09f501c863e05b71d9cf01adc7c39138   parcel 2018-10-11 19:57:58   
1    0x17820aee2388abdbc81b4b407140016d50c086df   parcel 2018-10-12 01:56:40   
2    0x5abdc3cb826fc0277d642c9fb52fa76fe3abb4e7   parcel 2018-10-12 02:20:57   
3    0xc1a1a63c331fc442bbbd04f32b923f8aa5f9f954   parcel 2018-10-12 04:42:01   
4    0x0ed1c03d04c1fbb33a0f25ef6147ab5a73d811a7   parcel 2018-10-12 05:22:18   
..                                          ...      ...                 ...   
995  0x54fd62c2a51de4e23e339c72c92436cf6387beac   parcel 2022-01-26 20:19:56   
996  0xf179344bbec75ad4522bd6b2c780b2e13b3469a2   parcel 2022-01-26 20:55:33   
997  0x4428a895806d7da2370f3375517fc1059339b7f1   parcel 2022-01-27 03:38:45   
998  0x7166f69486c7a525d1305da41ca6e65991e85717   parcel 2022-01-27 06:16:13   
999  0x5ce213893956bbf4249a7f8a079331280065eec6   parcel 2022-01-27 06:18:55   

                                       

In [28]:
#Extracting data with USD prices for daily prices conversion from MANA to USD

# initialize full_data for Query2 of Prices dataset
df3_fulldata = pd.DataFrame()
df1_data = pd.DataFrame()

# Set query (which uses text input to specify USD data)
query = '''query {
  tokenDayDatas(where:{token:"0x0f5d2fb29fb7d3cfee444a200298f468908cc942"}, orderBy: date, orderDirection: desc, first: 1000){
    token{
      symbol
    }
    priceUSD
    date
  }
}'''

# Make the request
r = requests.post(url2, json={'query': query})
#print(r.status_code)
#print(r.text)
    
# JSON adjustment
json_data = json.loads(r.text)
    
# extract JSON to convert to a dataframe
df1_data = json_data['data']['tokenDayDatas']
df3_fulldata = pd.DataFrame(df1_data)

#Removing NULLS - Data Cleaning
df2 = df3_fulldata.dropna()

#Converting from Unix timestamp to datetime
df2['date'] = pd.to_datetime(df2['date'],unit='s')

#Removing extra data from columns : Cleaning
df2['token'] = df2['token'].map(lambda x: str(x)[12:-2])

print(df2)

    token                               priceUSD       date
0    MANA    2.751290858071162006463054971598042 2022-02-23
1    MANA    2.641774203382024684484727812060281 2022-02-22
2    MANA    2.539409613838052042619624222447455 2022-02-21
3    MANA    2.728778533661602839800365732348586 2022-02-20
4    MANA    2.872108087149710793272651669382348 2022-02-19
..    ...                                    ...        ...
642  MANA  0.03994886997362385134304507643947843 2020-05-22
643  MANA  0.03659038590872996396210342513692342 2020-05-21
644  MANA  0.03765596530492163298229213491053662 2020-05-20
645  MANA   0.0365052011167876893752711366962339 2020-05-19
646  MANA                                      0 2020-05-18

[647 rows x 3 columns]


In [29]:
df1['date']=df1['date'].astype('datetime64')
df2['priceUSD'] = pd.to_numeric(df2['priceUSD'])
df1['price_MANA'] = pd.to_numeric(df1['price_MANA'])
print(df1.dtypes)
print(df2.dtypes)

buyer                 object
category              object
createdAt     datetime64[ns]
id                    object
owner                 object
status                object
tokenId               object
txHash                object
x                     object
y                     object
price_MANA           float64
date          datetime64[ns]
dtype: object
token               object
priceUSD           float64
date        datetime64[ns]
dtype: object


In [30]:
#Left join on the above 2 dataframes based on 'Date'
df = pd.merge(df1, df2[['priceUSD', 'date']], on='date', how='left')

#Calculating price in USD based on daily USD rate
df['price_USD'] = df['priceUSD'] * df['price_MANA']
df.sort_values(by=['createdAt'], inplace=True, ascending=False)

print(df)

                                           buyer category           createdAt  \
8999  0x5ce213893956bbf4249a7f8a079331280065eec6   parcel 2022-01-27 06:18:55   
8998  0x7166f69486c7a525d1305da41ca6e65991e85717   parcel 2022-01-27 06:16:13   
8997  0x4428a895806d7da2370f3375517fc1059339b7f1   parcel 2022-01-27 03:38:45   
8996  0xf179344bbec75ad4522bd6b2c780b2e13b3469a2   parcel 2022-01-26 20:55:33   
8995  0x54fd62c2a51de4e23e339c72c92436cf6387beac   parcel 2022-01-26 20:19:56   
...                                          ...      ...                 ...   
4     0x0ed1c03d04c1fbb33a0f25ef6147ab5a73d811a7   parcel 2018-10-12 05:22:18   
3     0xc1a1a63c331fc442bbbd04f32b923f8aa5f9f954   parcel 2018-10-12 04:42:01   
2     0x5abdc3cb826fc0277d642c9fb52fa76fe3abb4e7   parcel 2018-10-12 02:20:57   
1     0x17820aee2388abdbc81b4b407140016d50c086df   parcel 2018-10-12 01:56:40   
0     0x1b39eafd09f501c863e05b71d9cf01adc7c39138   parcel 2018-10-11 19:57:58   

                           

In [31]:
df.to_excel(r'C:\2021_NehalPersonal\Project\Decentraland_LATEST.xlsx', sheet_name='Data', index = False)